# Script Writer Workflow Using CrewAI


In this notebook, we demonstrate how to build and collaborating agentic AI system using **CrewAI**. The system integrates multiple agents, each with a specific role, to:
1. Gather current information from the internet about given topic.
2. Make a conversational script out of the information.

---
### Workflow Overview
1. Define tools to support the agents, such as web search capabilities.
2. Create two agents for data gathering and script creation.
3. Define tasks corresponding to these agents.
4. Set up a crew that executes the tasks sequentially.
5. Run the crew to achieve the desired outcome.

---
### Code Walkthrough
Below is the implementation.


### Step 1: Import Required Libraries and Define Tools

In [1]:
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()

# Current date for context
Now = datetime.now()
Today = Now.strftime("%d-%b-%Y")

In [2]:
# Web Search Tool
import os
from crewai_tools import EXASearchTool 

os.environ['EXA_API_KEY'] = os.getenv('EXA_API_KEY')

exa_search_tool = EXASearchTool ()

/Users/ishandutta/miniconda3/envs/agents/lib/python3.10/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


### Step 2: Define the Agents

In [3]:
from crewai import Agent
from crewai.llm import LLM

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
llm = LLM(model='gpt-4o', temperature=0.9)

## Agent designed to gather information about a topic from internet.
content_explorer = Agent (
    role = 'content explorer',
    goal = 'Gather and provide latest information about the topic from internet',
    llm=llm,
    verbose = True,
    backstory = ('You are an expert researcher, who can gather detailed information about a topic.\
                  Gather at least 10 information. Consider you are on : '+Today),
    tools = [exa_search_tool],
    cache = True,
    max_iter = 5
)

## Agent to create a script out of information.
## Back story sets the context for the agent.

script_writer = Agent (
    role = 'Script Writer',
    goal = 'With the details given to you create an interesting conversational script out of it',
    llm=llm,
    verbose = True,
    backstory = ('You are an expert in Shakesperean literature. You are very good in creating conversations with the given chain of information.\
                 Tell as a script in 200 words.\
                 Consider you are on '+Today),
)

os.environ['OTEL_SDK_DISABLED'] = "True"


### Step 3: Define the Tasks

In [4]:
from crewai import Task

# Task to gather Latest information
get_details = Task (
                      description = "Get latest, trending, interesting information and news about {topic}",
                      expected_output = "Latest news, interesting information and trivia about {topic}",
                      agent = content_explorer
                    )

## Task to create script.
create_a_script = Task (
                      description = "Considering the given details in time order make an interesting conversation",
                      expected_output = "A humorous conversation connecting key details",
                      agent = script_writer,
                      context = [get_details],
                      output_file = 'script.txt'
                    )



### Step 4: Set Up the Crew

In [5]:
from crewai import Crew, Process
from datetime import datetime

# Callback function to print a timestamp
def timestamp(Input):
    print(datetime.now())

# Define the crew with agents and tasks in sequential process
crew = Crew (
    agents = [content_explorer, script_writer],
    tasks = [get_details, create_a_script],
    verbose = True,
    Process = Process.sequential,
    step_callback = timestamp
)

### Step 5: Run the Crew and Observe Results

In [6]:
# Run the crew with a specific stock
result = crew.kickoff (inputs={'topic' : "Iran Isral conflict"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0d59da6a-a6be-4c86-a815-f0fd536b5aeb                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: content explorer                                                                                        │
│                                                                                                                 │
│  Task: Get latest, trending, interesting information and news about Iran Isral conflict                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

2025-07-28 20:46:16.912095

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: content explorer                                                                                        │
│                                                                                                                 │
│  Thought: I will use the EXASearchTool to gather the latest, trending, and interesting information and news     │
│  about the Iran-Israel conflict. I will search for articles and information published recently to ensure that   │
│  the details are up-to-date.                                                                                    │
│                                                                                                                 │
│  Using Tool: EXASearchTool                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

2025-07-28 20:46:42.320669

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Script Writer                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Scene: A quaint café in Verona. Benedick and Beatrice, the quintessential witty banterers from               │
│  Shakespeare’s "Much Ado About Nothing," are enjoying an afternoon tea. It's the 28th of July, 2025. Around     │
│  them, patrons flip through newspapers and scroll on phones, following the latest international news.**         │
│                                                                                                                 │
│  **Benedick:** (sipping his tea) Good morrow, sweet Beatrice. The world spins madly on, and methinks even the   │
│  bards of old would struggle to pen these tales.                                                                │
│                                                                                                                 │
│  **Beatrice:** (raising an eyebrow) Oh, dost thou speak of the merriment or the madness, Benedick? Pray,        │
│  enlighten me.                                                                                                  │
│                                                                                                                 │
│  **Benedick:** (leaning in conspiratorially) 'Tis the ever-tumultuous tale of Iran and Israel, my dear. A saga  │
│  most complex, where diplomacy waltzes with daggers drawn.                                                      │
│                                                                                                                 │
│  **Beatrice:** (feigning shock) Dost thou jest? I thought they were all for peace talks over pistols.           │
│                                                                                                                 │
│  **Benedick:** (chuckling) Aye, but like a fool attempting poetry, words oft go awry. Even now, wise men argue  │
│  as to who should fold first in this delicate dance.                                                            │
│                                                                                                                 │
│  **Beatrice:** (smirking) Such is the world, Benedick. When peace is near, conflict hides around the corner     │
│  like a cat ready to pounce.                                                                                    │
│                                                                                                                 │
│  **Benedick:** (with a theatrical sigh) Verily, it seems the conflict is a never-ending play. Were we but       │
│  jesters in a grand theatre, dearest Beatrice, our audience might never tire.                                   │
│                                                                                                                 │
│  **Beatrice:** (raising her cup) Here's to hoping they find their soliloquy of peace soon, for this play has    │
│  run long enough.                                                                                               │
│                                                                                                                 │
│  **Benedick:** (raising his cup in return) A toast to peace, and to wit sharper than any sword.                 │
│                                                                                                                 │
│  (They clink their cups, and laughter fills the air.)  

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 05566970-fe58-414e-b04f-0e5b36b7534a                                                                     │
│  Agent: Script Writer                                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0d59da6a-a6be-4c86-a815-f0fd536b5aeb                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Scene: A quaint café in Verona. Benedick and Beatrice, the quintessential witty banterers      │
│  from Shakespeare’s "Much Ado About Nothing," are enjoying an afternoon tea. It's the 28th of July, 2025.       │
│  Around them, patrons flip through newspapers and scroll on phones, following the latest international news.**  │
│                                                                                                                 │
│  **Benedick:** (sipping his tea) Good morrow, sweet Beatrice. The world spins madly on, and methinks even the   │
│  bards of old would struggle to pen these tales.                                                                │
│                                                                                                                 │
│  **Beatrice:** (raising an eyebrow) Oh, dost thou speak of the merriment or the madness, Benedick? Pray,        │
│  enlighten me.                                                                                                  │
│                                                                                                                 │
│  **Benedick:** (leaning in conspiratorially) 'Tis the ever-tumultuous tale of Iran and Israel, my dear. A saga  │
│  most complex, where diplomacy waltzes with daggers drawn.                                                      │
│                                                                                                                 │
│  **Beatrice:** (feigning shock) Dost thou jest? I thought they were all for peace talks over pistols.           │
│                                                                                                                 │
│  **Benedick:** (chuckling) Aye, but like a fool attempting poetry, words oft go awry. Even now, wise men argue  │
│  as to who should fold first in this delicate dance.                                                            │
│                                                                                                                 │
│  **Beatrice:** (smirking) Such is the world, Benedick. When peace is near, conflict hides around the corner     │
│  like a cat ready to pounce.                                                                                    │
│                                                                                                                 │
│  **Benedick:** (with a theatrical sigh) Verily, it seems the conflict is a never-ending play. Were we but       │
│  jesters in a grand theatre, dearest Beatrice, our audience might never tire.                                   │
│                                                                                                                 │
│  **Beatrice:** (raising her cup) Here's to hoping they find their soliloquy of peace soon, for this play has    │
│  run long enough.                                                                                               │
│                                                                                                                 │
│  **Benedick:** (raising his cup in return) A toast to peace, and to wit sharper than any sword.                 │
│                                                       


---
### Conclusion
This workflow demonstrates how to integrate multiple agents to perform tasks sequentially in a context. It showcases the power of CrewAI for building AI-driven collaborative solutions.
